<a href="https://colab.research.google.com/github/amartya1056/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# API to fetch the dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 204MB/s]
100% 80.9M/80.9M [00:00<00:00, 183MB/s]


In [ ]:
# Extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing


In [ ]:
# Loading the Data from CSV file to pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
#Printing the first five rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# Naming the Columns and reading the dataset again
column_names= ['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Counting the Number of Missing Values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the Target "4" to "1"

In [ ]:
twitter_data.replace({'target' : {4:1}}, inplace=True)

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 ----> Negative Tweet
1 ----> Postive Tweet


Stemming

Stemming is the process of reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming (content):
  stemmed_content = re.sub('[^a-zA-z] ',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()

  #Stemming Operation
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

50 Minutes Runtime for performing the stemming Operation on 1.6 million tweets

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",@switchfoot http://twitpic.com/2y1zl awww that...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't updat facebook text it.. might cri...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan dive mani time ball manag save 50 re...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",@nationwideclass behav mad can't see


In [ ]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl awww that...
1          upset can't updat facebook text it.. might cri...
2          @kenichan dive mani time ball manag save 50 re...
3                            whole bodi feel itchi like fire
4                       @nationwideclass behav mad can't see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb.com cool hear old walt interview â htt...
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time!! tupac amar...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# Seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl awww that' bummer shoulda got david carr third day ;d"
 "upset can't updat facebook text it.. might cri result school today also blah!"
 '@kenichan dive mani time ball manag save 50 rest go bound' ...
 'readi mojo makeov ask detail'
 'happi 38th birthday boo alll time!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the Data into training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' '@hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day can't wait hear guy store tomorrow!"
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['@mmangen fine much time chat twitter hubbi back summer &amp tend domin free time'
 'ah may show w ruth kim &amp geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini4 nevertheless hooray 470 member wonder safe trip' 'feel well'
 '@supersandro !!! thank']


In [ ]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 532555)	0.2725523328084958
  (0, 440025)	0.35850162814522984
  (0, 246248)	0.5251991902119445
  (0, 161790)	0.37629494770304556
  (0, 302999)	0.42196106798918537
  (0, 539546)	0.4491197930766051
  (1, 219200)	1.0
  (2, 161790)	0.45964634999290466
  (2, 178861)	0.18941350482831693
  (2, 499682)	0.18668085882569893
  (2, 184689)	0.290741456676746
  (2, 498719)	0.3207738710121683
  (2, 529067)	0.32893271312659217
  (2, 125562)	0.3129631648817351
  (2, 539947)	0.3356341982946679
  (2, 339604)	0.24126158547956847
  (2, 501761)	0.15224909998344602
  (2, 238240)	0.16164643282513708
  (2, 208108)	0.18762581654097307
  (2, 187801)	0.20251086097730572
  (2, 364939)	0.16747522071152948
  (3, 498719)	0.2899440868521427
  (3, 216981)	0.44646978067660353
  (3, 209274)	0.27801537419973615
  (3, 100777)	0.5189390326964144
  :	:
  (1279996, 529554)	0.26890659738990275
  (1279996, 480318)	0.21772782909324642
  (1279996, 460922)	0.35595319627404676
  (1279996, 306529)	0.5370998202985965
  (1279996,

In [ ]:
print(X_test)

  (0, 54088)	0.17118096696651894
  (0, 72606)	0.161772940193475
  (0, 114090)	0.26733343211281335
  (0, 158095)	0.3691042261143041
  (0, 187861)	0.2544233295775757
  (0, 194313)	0.23720131845175768
  (0, 230951)	0.2836687086987593
  (0, 344594)	0.4515138995739004
  (0, 353923)	0.1774822056753894
  (0, 478293)	0.2193613999350176
  (0, 490467)	0.3475247266418872
  (0, 501761)	0.3147238340189874
  (0, 515134)	0.17902284870117902
  (1, 44284)	0.30399480836208426
  (1, 54088)	0.20821424594665175
  (1, 202368)	0.5714146731177951
  (1, 283096)	0.4070732819545278
  (1, 328255)	0.2840923671284761
  (1, 432436)	0.4818916829327709
  (1, 453146)	0.24306898526504928
  (2, 62607)	0.3531860406495728
  (2, 76221)	0.37903231605173976
  (2, 139383)	0.3629427319713099
  (2, 244269)	0.588213814848275
  (2, 328345)	0.25720502008796964
  :	:
  (319995, 160122)	0.2936347927285973
  (319995, 161873)	0.26569017851516924
  (319995, 213353)	0.24305420588385093
  (319995, 278431)	0.23539770342178995
  (319995, 30

Training the Logistic Regression Model

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [ ]:
# Accuracy Score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score on the training data', training_data_accuracy)

Accuracy score on the training data 0.80735


In [ ]:
# Accuracy Score on test data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score on the test data', testing_data_accuracy)

Accuracy score on the test data 0.780384375


Saving the Trained Model

In [ ]:
import pickle

In [ ]:
file= 'trained_model.sav'
pickle.dump(model, open(file,'wb')) #wb means writing the file in binary

Using the Saved Model for Future Predictions

In [ ]:
#loading the saved model
loaded_model=pickle.load(open('trained_model.sav','rb')) #rb means reading the file in binary

In [ ]:
X_new = X_test[200]
print(Y_test[200])
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The Tweet is Negative')
else:
  print('The Tweet is Positive')

1
[1]
The Tweet is Positive


In [ ]:
X_new = X_test[3]
print(Y_test[3])
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The Tweet is Negative')
else:
  print('The Tweet is Positive')

0
[0]
The Tweet is Negative
